In [13]:
import pandas as pd
train_data = pd.read_csv("clean_wiki.txt")

text = train_data.to_string()

# preprocess the text
data_new = text_cleaner(text)

from math import floor

def get_training_and_testing_sets(file_list):
    split = 0.95
    split_index = floor(len(file_list) * split)
    training = file_list[:split_index]
    testing = file_list[split_index:]
    return training, testing

training, testing = get_training_and_testing_sets(data_new)

In [14]:
print(len(training))
print(len(testing))

8774732
461828


In [15]:
import regex

def text_cleaner(text):
    # lower case text
    newString = text.lower()
    # adding space in place of punctuation between two words e.g: "...redaktə edilib.Bu", so it can be split later.
    newString = regex.sub((r"(?<=\w)(\W)(?=\w)"), ' ', newString)
    # remove punctuations
    newString = regex.sub((r"[^\A\p{L}+\z ]"), '', newString) # matches unicode characters 
    # remove short words x_X
    long_words = []
    for i in newString.split():
        if len(i) >= 3:
            long_words.append(i)
    return (" ".join(long_words)).strip()

In [16]:

from collections import Counter
from nltk.util import ngrams
import string
import nltk
from nltk import SimpleGoodTuringProbDist
from math import log

class MarkovChain:

    def __init__(self):
        self.memory = {}

    def _learn_key(self, *key, value):
        if key not in self.memory:
            self.memory[key] = []

        self.memory[key].append(value)
        
    def _learn_key1(self, *key):
        if key not in self.memory:
            self.memory[key] = []

        self.memory[key].append(key)

    def train1(self, text):
        
        tokens = text.split(" ")
        unigrams = [(tokens[i]) for i in range(0, len(tokens))]
        combo_count = Counter(unigrams)
        for unigram in unigrams:
            self._learn_key1(unigram[0])
        return combo_count.most_common(20)
    def train2(self, text):
        
        tokens = text.split(" ")
        bigrams = [(tokens[i], tokens[i + 1]) for i in range(0, len(tokens) - 1)]
        combo_count = Counter(bigrams)
        for bigram in bigrams:
            self._learn_key(bigram[0], value = bigram[1])
        return combo_count.most_common(20)
    
    def train3(self, text):
        tokens = text.split(" ")
        trigrams = [(tokens[i], tokens[i + 1], tokens[i + 2]) for i in range(0, len(tokens) - 2)]
        tri_count = Counter(trigrams)
        # pprint(trigrams)
        for trigram in trigrams:
            self._learn_key(trigram[0], trigram[1], value = trigram[2])
        return tri_count.most_common(30)
    def train4(self, text):
        tokens = text.split(" ")
        quadgrams = [(tokens[i], tokens[i + 1], tokens[i + 2], tokens[i + 3]) for i in range(0, len(tokens) - 3)]
        quad_count = Counter(quadgrams)
        # pprint(quadgrams)
        for quadgram in quadgrams:
            self._learn_key(quadgram[0], quadgram[1], quadgram[2], value = quadgram[3])
        return quad_count.most_common(30)
    
    def train5(self, text):
        tokens = text.split(" ")
        fivegrams = [(tokens[i], tokens[i + 1], tokens[i + 2], tokens[i + 3], tokens[i + 4]) for i in range(0, len(tokens) - 4)]
        five_count = Counter(fivegrams)
        # pprint(fivegrams)
        for fivegram in fivegrams:
            self._learn_key(fivegram[0], fivegram[1], fivegram[2],fivegram[3], value = fivegram[4])
        return five_count.most_common(30)

    def next(self, *current_state):
        next_possible = self.memory.get(current_state)

        if not next_possible:
            next_possible = self.memory.keys()

        cnt = Counter(next_possible) 
        
#         next_possible = cnt.most_common(1)[0]
        next_possible = cnt.most_common(1)[0][0]
    
        return next_possible
    
    def _unigram_prob_with_add1smoothing(self, text, word):
        freq_1gram= nltk.FreqDist(text)
        len_train= len(text)
        vocab= len(set(text))
        return (freq_1gram[word]+1)/(len_train+vocab)
    
    def _bigram_prob_with_add1smoothing(self,text,word1, word2):
        tokens = text.split(" ")
        bigrams = [(tokens[i], tokens[i + 1]) for i in range(0, len(tokens) - 1)]
        cfreq_2gram = nltk.ConditionalFreqDist(bigrams)
        cprob_2gram = nltk.ConditionalProbDist(cfreq_2gram, nltk.MLEProbDist)
        cprob_2gram_add1=float((((1+cfreq_2gram[word1][word2])/(len(cfreq_2gram)+sum(cfreq_2gram[word1].values())))))
        return cprob_2gram_add1
       
    def _trigram_prob_with_add1smoothing(self,text,w1, w2, w3):
        tokens = text.split(" ")
        trigrams = [(tokens[i], tokens[i + 1], tokens[i + 2]) for i in range(0, len(tokens) - 2)]
        trigrams_as_bigrams=[]
        trigrams_as_bigrams.extend([((t[0],t[1]), t[2]) for t in trigrams])
        cfreq_3gram = nltk.ConditionalFreqDist(trigrams_as_bigrams)
        cprob_3gram = nltk.ConditionalProbDist(cfreq_3gram, nltk.MLEProbDist)
        cprob_3gram_add1=(((1+cfreq_3gram[(w1,w2)][w3])/(len(cfreq_3gram)+sum(cfreq_3gram[(w1,w2)].values()))))
        return cprob_3gram_add1
        
    def _fourgram_prob_with_add1smoothing(self,text,w1, w2, w3,w4):
        tokens = text.split(" ")
        fourgrams = [(tokens[i], tokens[i + 1], tokens[i + 2],tokens[i+3]) for i in range(0, len(tokens) - 3)]
        fourgrams_as_trigrams=[]
        fourgrams_as_trigrams.extend([((t[0],t[1],t[2]),t[3]) for t in fourgrams])
        cfreq_4gram = nltk.ConditionalFreqDist(fourgrams_as_trigrams)
        cprob_4gram = nltk.ConditionalProbDist(cfreq_4gram, nltk.MLEProbDist)
        cprob_4gram_add1=(((1+cfreq_4gram[(w1,w2,w3)][w4])/(len(cfreq_4gram)+sum(cfreq_4gram[(w1,w2,w3)].values()))))
        return cprob_4gram_add1
    
    def _fivegram_prob_with_add1smoothing(self,text,w1, w2, w3,w4,w5):
        tokens = text.split(" ")
        fourgrams = [(tokens[i], tokens[i + 1], tokens[i + 2],tokens[i+3],tokens[i+4]) for i in range(0, len(tokens) - 4)]
        fivegrams_as_fourgrams=[]
        fivegrams_as_fourgrams.extend([((t[0],t[1],t[2],t[3]),t[4]) for t in fivegrams])
        cfreq_5gram = nltk.ConditionalFreqDist(fivegrams_as_fourgrams)
        cprob_5gram = nltk.ConditionalProbDist(cfreq_5gram, nltk.MLEProbDist)
        cprob_5gram_add1=(((1+cfreq_5gram[(w1,w2,w3,w4)][w5])/(len(cfreq_5gram)+sum(cfreq_5gram[(w1,w2,w3,w4)].values()))))
        return cprob_5gram_add1
    
    def _entropy(self,n, text):
        e = 0.0
#         text = ["<s>"] + text + ["</s>"]
        for i in range(n - 1, len(text)):
            context = text[i - n + 1:i]
            token = text[i]
            #print(str(context)+"    "+token)
            e += self._logprob(text,token, context)
        return e / float(len(text) - (n - 1))


    def _logprob(self,text,word, context):
        if len(context)==0:
            p=self._unigram_prob_with_add1smoothing(text,word)
        elif len(context)==1:
            p=self._bigram_prob_with_add1smoothing(text,context[0], word)
        elif len(context)==2:
            p=self._trigram_prob_with_add1smoothing(text,context[0], context[1], word)
        else:
            p=self._fourgram_prob_with_add1smoothing(text,context[0], context[1],context[2],word)
        return -p*log(p , 2)


    def perplexity(self,n, text):
          return pow(2.0, self._entropy(n, text))

In [20]:
m = MarkovChain()
m.train1(training)

[('kateqoriya', 56028),
 ('görə', 17280),
 ('aid', 14628),
 ('azərbaycan', 11717),
 ('növü', 11491),
 ('cinsinə', 11158),
 ('bitki', 10693),
 ('istinadlar', 10381),
 ('mənbə', 10224),
 ('fəsiləsinin', 10072),
 ('ildə', 8037),
 ('ölkələrinə', 6909),
 ('xarici', 6161),
 ('keçidlər', 5516),
 ('bax', 5464),
 ('kənd', 4761),
 ('əhalisi', 4513),
 ('olan', 4424),
 ('xal', 3700),
 ('həmçinin', 3684)]

In [ ]:
m.perplexity(1,testing)

In [19]:
m.train2(training)


[(('cinsinə', 'aid'), 11150),
 (('aid', 'bitki'), 10370),
 (('bitki', 'növü'), 9984),
 (('fəsiləsinin', 'cinsinə'), 9944),
 (('növü', 'mənbə'), 7755),
 (('ölkələrinə', 'görə'), 6806),
 (('istinadlar', 'kateqoriya'), 5816),
 (('kateqoriya', 'ölkələrinə'), 5593),
 (('xarici', 'keçidlər'), 5491),
 (('mənbə', 'fəsiləsinin'), 5041),
 (('həmçinin', 'bax'), 3480),
 (('azərbaycan', 'respublikasının'), 2328),
 (('kateqoriya', 'azərbaycan'), 2309),
 (('illərinə', 'görə'), 2166),
 (('mənbə', 'xarici'), 2132),
 (('daxil', 'olan'), 2007),
 (('növü', 'istinadlar'), 2002),
 (('müzakirə', 'redaktə'), 1922),
 (('bax', 'müzakirə'), 1920),
 (('azərbaycan', 'ostanının'), 1878)]

In [ ]:
m.perplexity(2,testing)

In [21]:
m.train3(training)

[(('aid', 'bitki', 'növü'), 9963),
 (('fəsiləsinin', 'cinsinə', 'aid'), 9944),
 (('cinsinə', 'aid', 'bitki'), 9936),
 (('bitki', 'növü', 'mənbə'), 6907),
 (('kateqoriya', 'ölkələrinə', 'görə'), 5531),
 (('mənbə', 'fəsiləsinin', 'cinsinə'), 5041),
 (('növü', 'mənbə', 'fəsiləsinin'), 4897),
 (('mənbə', 'xarici', 'keçidlər'), 2131),
 (('bax', 'müzakirə', 'redaktə'), 1920),
 (('növü', 'mənbə', 'xarici'), 1919),
 (('bitki', 'növü', 'istinadlar'), 1884),
 (('siyahısı', 'səhləbkimilərə', 'aid'), 1832),
 (('xarici', 'keçidlər', 'bulbofilium'), 1831),
 (('keçidlər', 'bulbofilium', 'yoxlama'), 1831),
 (('bulbofilium', 'yoxlama', 'siyahısı'), 1831),
 (('yoxlama', 'siyahısı', 'səhləbkimilərə'), 1831),
 (('səhləbkimilərə', 'aid', 'növlərin'), 1831),
 (('aid', 'növlərin', 'internet'), 1831),
 (('növlərin', 'internet', 'foto'), 1831),
 (('internet', 'foto', 'ensiklopediyası'), 1825),
 (('foto', 'ensiklopediyası', 'fəsiləsinin'), 1813),
 (('ensiklopediyası', 'fəsiləsinin', 'cinsinə'), 1813),
 (('istin

In [ ]:
m.perplexity(3,testing)

In [22]:
m.train4(training)

[(('cinsinə', 'aid', 'bitki', 'növü'), 9935),
 (('fəsiləsinin', 'cinsinə', 'aid', 'bitki'), 9854),
 (('aid', 'bitki', 'növü', 'mənbə'), 6906),
 (('mənbə', 'fəsiləsinin', 'cinsinə', 'aid'), 5041),
 (('növü', 'mənbə', 'fəsiləsinin', 'cinsinə'), 4897),
 (('bitki', 'növü', 'mənbə', 'fəsiləsinin'), 4841),
 (('növü', 'mənbə', 'xarici', 'keçidlər'), 1919),
 (('aid', 'bitki', 'növü', 'istinadlar'), 1880),
 (('bitki', 'növü', 'mənbə', 'xarici'), 1860),
 (('xarici', 'keçidlər', 'bulbofilium', 'yoxlama'), 1831),
 (('keçidlər', 'bulbofilium', 'yoxlama', 'siyahısı'), 1831),
 (('bulbofilium', 'yoxlama', 'siyahısı', 'səhləbkimilərə'), 1831),
 (('yoxlama', 'siyahısı', 'səhləbkimilərə', 'aid'), 1831),
 (('siyahısı', 'səhləbkimilərə', 'aid', 'növlərin'), 1831),
 (('səhləbkimilərə', 'aid', 'növlərin', 'internet'), 1831),
 (('aid', 'növlərin', 'internet', 'foto'), 1831),
 (('növlərin', 'internet', 'foto', 'ensiklopediyası'), 1825),
 (('internet', 'foto', 'ensiklopediyası', 'fəsiləsinin'), 1813),
 (('foto'

In [ ]:
m.perplexity(4,testing)

In [23]:
m.train5(training)

[(('fəsiləsinin', 'cinsinə', 'aid', 'bitki', 'növü'), 9853),
 (('cinsinə', 'aid', 'bitki', 'növü', 'mənbə'), 6906),
 (('mənbə', 'fəsiləsinin', 'cinsinə', 'aid', 'bitki'), 4986),
 (('növü', 'mənbə', 'fəsiləsinin', 'cinsinə', 'aid'), 4897),
 (('bitki', 'növü', 'mənbə', 'fəsiləsinin', 'cinsinə'), 4841),
 (('aid', 'bitki', 'növü', 'mənbə', 'fəsiləsinin'), 4840),
 (('cinsinə', 'aid', 'bitki', 'növü', 'istinadlar'), 1880),
 (('aid', 'bitki', 'növü', 'mənbə', 'xarici'), 1860),
 (('bitki', 'növü', 'mənbə', 'xarici', 'keçidlər'), 1860),
 (('xarici', 'keçidlər', 'bulbofilium', 'yoxlama', 'siyahısı'), 1831),
 (('keçidlər', 'bulbofilium', 'yoxlama', 'siyahısı', 'səhləbkimilərə'), 1831),
 (('bulbofilium', 'yoxlama', 'siyahısı', 'səhləbkimilərə', 'aid'), 1831),
 (('yoxlama', 'siyahısı', 'səhləbkimilərə', 'aid', 'növlərin'), 1831),
 (('siyahısı', 'səhləbkimilərə', 'aid', 'növlərin', 'internet'), 1831),
 (('səhləbkimilərə', 'aid', 'növlərin', 'internet', 'foto'), 1831),
 (('aid', 'növlərin', 'internet

In [ ]:
m.perplexity(5,testing)